# 중고차 가격예측 

In [77]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('max_rows',None)
pd.set_option('max_columns',None)


X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv')
x_label = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv')

In [78]:
display(X_train.head())
display(y_train.head())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [79]:
df = pd.merge(X_train, y_train, on='carID')
#display(df.head())
X = df.drop('price',axis=1)
y = df.price
# print(y.head())
print(X.info())
display(X.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4960 entries, 0 to 4959
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carID         4960 non-null   int64  
 1   brand         4960 non-null   object 
 2   model         4960 non-null   object 
 3   year          4960 non-null   int64  
 4   transmission  4960 non-null   object 
 5   mileage       4960 non-null   int64  
 6   fuelType      4960 non-null   object 
 7   tax           4960 non-null   float64
 8   mpg           4960 non-null   float64
 9   engineSize    4960 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 426.2+ KB
None


,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


In [80]:
print(df.year.unique())

[2019 2015 2016 2020 2012 2018 2017 2009 2013 2006 2011 2014 2008 1997
 2001 2010 2005 2004 2007 2000 2003 2002 1999 1998]


In [81]:
# string변수 labelencoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def mk_LE(df):
    
    obj = df.select_dtypes(include='object').columns
    
    for col in obj:
        df[col] = le.fit_transform(df[col])
        
    return df    

In [82]:
X_LE = mk_LE(X)
# display(X_LE.head())
x_test = mk_LE(x_test)

In [83]:
X_LE.drop('carID',axis=1,inplace=True)
x_test.drop('carID',axis=1,inplace=True)

## year을 숫자로 보지 않고 labelencoding으로 전처리하기

In [84]:
X_LE['year'] = le.fit_transform(X_LE['year']) #년도는 숫자로 인식하기보다 특정문자로 인식하는걸로 생각해보기로함
x_test['year'] =le.fit_transform(x_test['year'])

In [85]:
X_LE.head()

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,3,69,22,3,4223,0,145.0,39.8,2.2
1,7,30,18,1,47870,0,125.0,60.1,2.0
2,0,58,22,0,5151,4,145.0,29.1,2.9
3,8,71,19,0,20423,0,30.0,57.6,2.0
4,5,70,23,3,3569,4,145.0,47.1,1.0


In [86]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def MinMax_scaler(df):
    cols = ['mileage','tax','mpg','engineSize']
    for col in cols: 
        df[col] = scaler.fit_transform(df[[col]])
        
    return df


In [87]:
X_LE = MinMax_scaler(X_LE)
x_test = MinMax_scaler(x_test)

In [88]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_LE,y, test_size=0.2,random_state=42)

print(X_train.shape)

(3968, 9)


In [33]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X_train,y_train)
pred=lr.predict(X_test)


In [34]:
from sklearn.metrics import r2_score

r2_score(y_test,pred)

0.6745707091671806

In [89]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
pred = rfr.predict(X_test)

from sklearn.metrics import r2_score
r2_score(y_test,pred)

0.9600810689094786

In [39]:
print(len(pred), len(x_label),len(y_test))

992 2672 992


In [42]:
a = pd.DataFrame({'y_test':y_test,'pred':pred}).reset_index(drop=True)

In [44]:
a.head()

,y_test,pred
0,20500,19754.83
1,15199,17475.89
2,18450,22681.39
3,12995,11883.88
4,19750,23776.54


In [46]:
len(x_test), len(y)

(2672, 4960)

In [90]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_LE,y)
prediction = rfr.predict(x_test)

In [91]:
print(r2_score(x_label['price'], prediction))

0.8495392326723333


In [92]:
answer = pd.DataFrame({'carID':x_label.carID,'real':x_label.price,'pred':prediction})
answer.head()


,carID,real,pred
0,12000,38000,38217.23
1,12001,23495,19223.98
2,12004,59999,45756.47
3,12013,16713,14047.21
4,12017,46000,42537.31


## year을 숫자로 보고 전처리하기

In [66]:
display(X_LE.head(3))
display(x_test.head(3))

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,3,69,2019,3,4223,0,145.0,39.8,2.2
1,7,30,2015,1,47870,0,125.0,60.1,2.0
2,0,58,2019,0,5151,4,145.0,29.1,2.9


,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,4,28,2017,0,12046,0,150.0,37.2,3.0
1,8,6,2017,0,37683,0,260.0,36.2,3.0
2,4,28,2019,0,10000,0,145.0,34.0,3.0


In [68]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def MinMax_scaler(df):
    
    cols = ['year','mileage','tax','mpg','engineSize']
            
    for col in cols:
            df[col]=scaler.fit_transform(df[[col]])
    return df

In [69]:
X_LE = MinMax_scaler(X_LE)
x_test = MinMax_scaler(x_test)

In [70]:
display(x_test.head())

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,4,28,0.94,0,0.072103,0,0.258621,0.073504,0.476190
1,8,6,0.94,0,0.225569,0,0.448276,0.071368,0.476190
2,4,28,0.98,0,0.059855,0,0.250000,0.066667,0.476190
3,5,69,0.98,1,0.019491,4,0.250000,0.100000,0.158730
4,0,59,0.90,3,0.125595,4,0.560345,0.056838,0.634921


In [71]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X_LE,y, test_size=0.2, random_state=42)

In [72]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()

rfr.fit(X_train,y_train)
y_pred = rfr.predict(X_test)

In [73]:
from sklearn.metrics import r2_score

print(r2_score(y_test,y_pred))

0.9621608744625785


In [74]:
rfr.fit(X_LE,y)
prediction = rfr.predict(x_test)


In [76]:
print(r2_score(x_label.price,prediction))

0.8122947914344484
